### Import, mount, and configure

In [3]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import sklearn
import statsmodels
import tensorflow as tf
import tqdm
from google.colab import drive

In [4]:
drive.mount('/content/drive')

DRIVE_ROOT = Path('/content/drive/MyDrive')

PROJECT_ROOT = DRIVE_ROOT / 'projects/btc'
PROJECT_DATA_DIR = PROJECT_ROOT / 'data'
PROJECT_CLEANED_DIR = PROJECT_DATA_DIR / 'cleaned'

BTC_CLEANED_CSV_FILENAME = 'btc_274_cleaned.csv'

BTC_CLEANED_FILEPATH = PROJECT_CLEANED_DIR / BTC_CLEANED_CSV_FILENAME

Mounted at /content/drive


In [5]:
if 'google.colab' in sys.modules:
    %pip install -q -U keras_tuner

import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 6.3 MB/s eta 0:00:00


In [6]:
%pip install finta

In [7]:
project_root = os.path.abspath(PROJECT_ROOT)

if project_root not in sys.path:
  sys.path.append(project_root)

print(f'Project root "{project_root}" added to sys.path.')

Project root "/content/drive/MyDrive/projects/btc" added to sys.path.


In [8]:
from src.analysis import *
from src.data_loader import *
from src.data_splitter import *
from src.model import *
from src.preprocessing import *
from src.utils import *
from src.visualization import *

### Load the cleaned dataset, and resample it

In [9]:
# A missing date(2025-03-15) exists in the test dataset.
X_btc_cleaned = load_btc_dataset(BTC_CLEANED_FILEPATH, 'date', True)
btc_resampled = resample_btc_data(X_btc_cleaned)


Loading dataset from: btc_274_cleaned.csv...
Dataset loaded successfully.

--- Resampling BTC Data to Multiple Timeframes ---
Resampling to hourly frequency...
Resampling to daily frequency...
Resampling to weekly frequency...
Resampling to monthly frequency...
--- BTC Data Resampling Complete ---


### Split the resampled datasets into a training set, a validation set, and a test set of respective timeframe, and take returns

In [11]:
for X_btc_resampled in btc_resampled.values():
    X_btc_resampled = calculate_returns(X_btc_resampled, 'close')

In [12]:
X_btc_daily_train, X_btc_daily_valid, X_btc_daily_test = split_btc_dataset(btc_resampled['daily'], 'Day')
X_btc_weekly_train, X_btc_weekly_valid, X_btc_weekly_test = split_btc_dataset(btc_resampled['weekly'], 'Week')


--- Splitting BTC Dataset to Training, Validation, and Test Set (Day) ---
--- BTC Dataset Splitting Complete ---

--- Splitting BTC Dataset to Training, Validation, and Test Set (Week) ---
--- BTC Dataset Splitting Complete ---


### Run the ADF test on datasets at multiple frequencies to verify stationarity and seasonality

In [13]:
run_adf_test(X_btc_daily_train, 'returns_close', 'BTC Daily Closing Returns')
run_adf_test(X_btc_weekly_train, 'returns_close', 'BTC Weekly Closing Returns')

= ADF Test: BTC Daily Closing Returns =
ADF Statistics: -10.1710
p-value: 0.0000
Critical Values:
	1%:-3.4320
	5%:-2.8623
	10%:-2.5672
Conclusion: The p-value is less than or equal to 0.05. The data is likely stationary and seasonal.

= ADF Test: BTC Weekly Closing Returns =
ADF Statistics: -11.0168
p-value: 0.0000
Critical Values:
	1%:-3.4421
	5%:-2.8667
	10%:-2.5695
Conclusion: The p-value is less than or equal to 0.05. The data is likely stationary and seasonal.



### GRU Model Evaluation and Final Forecast (1-Month)

In [14]:
tf.random.set_seed(42)

In [15]:
FEATURE_COLS = ['open', 'high', 'low', 'close', 'volume']
TARGET_COL = 'close'
INPUT_WINDOW = 270
TARGET_WINDOW = 30
FORECAST_HORIZON = 30

In [16]:
naive_metrics, naive_pred = run_naive_model(X_btc_daily_valid, 'close', 30, 'Days', 'Validation')

= Naive Model Evaluation on Validation Set (Horizon: 30 Days) =
--- Skipping Naive Model Fitting ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 10.5643%
- Directional Accuracy (DA): 54.8611%

--- Generating Final Naive Forecast ---
- Forecast for 2024-03-15: $51475.00


In [17]:
gru_metrics, gru_pred = run_gru_model(X_btc_daily_train, X_btc_daily_valid, X_btc_daily_test,
                                      FEATURE_COLS, TARGET_COL, FORECAST_HORIZON,
                                      INPUT_WINDOW, TARGET_WINDOW,
                                      'Days', 'Validation',
                                      naive_metrics, 'direct')

Trial 15 Complete [00h 01m 29s]
val_mae: 0.3603944778442383

Best val_mae So Far: 0.3602541983127594
Total elapsed time: 05h 43m 35s

LSTM hyperparameter tuning complete:
- n_conv_layers: 2
- kernel_size: 5
- n_rnn_layers: 1
- use_l2: True
- dropout_rate: 0.0
- learning_rate: 0.005218086982564764
- optimizer: nadam
- clipnorm: 0.8
- filters_0: 192
- units_0: 192
- filters_1: 64
- units_1: 224
- filters_2: 96
- units_2: 32
- l2_rate: 0.0025124079357885637

--- Fitting GRU Model ---
Epoch 1/100
113/113 - 5s - 45ms/step - loss: 0.7692 - mae: 0.6642 - val_loss: 0.1692 - val_mae: 0.3678 - learning_rate: 0.0052
Epoch 2/100
113/113 - 2s - 16ms/step - loss: 0.3412 - mae: 0.6269 - val_loss: 0.1345 - val_mae: 0.3607 - learning_rate: 0.0052
Epoch 3/100
113/113 - 2s - 16ms/step - loss: 0.3331 - mae: 0.6288 - val_loss: 0.1342 - val_mae: 0.3622 - learning_rate: 0.0052
Epoch 4/100
113/113 - 2s - 16ms/step - loss: 0.3322 - mae: 0.6286 - val_loss: 0.1347 - val_mae: 0.3640 - learning_rate: 0.0052
Epoch 

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, None, 192)      │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, None, 64)       │        61,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, None, 192)      │       148,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, None, 192)      │           384 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 30)       │         5,790 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221,278 (864.37 KB)

 Trainable params: 221,278 (864.37 KB)

 Non-trainable params: 0 (0.00 B)

--- Fitting GRU Model ---
--- Preparing Data for GRU ---
Epoch 1/100
128/128 - 6s - 44ms/step - loss: 0.7162 - mae: 0.6584 - val_loss: 0.1982 - val_mae: 0.4526 - learning_rate: 0.0052
Epoch 2/100
128/128 - 2s - 16ms/step - loss: 0.3386 - mae: 0.6290 - val_loss: 0.1888 - val_mae: 0.4734 - learning_rate: 0.0052
Epoch 3/100
128/128 - 2s - 16ms/step - loss: 0.3335 - mae: 0.6290 - val_loss: 0.1810 - val_mae: 0.4591 - learning_rate: 0.0052
Epoch 4/100
128/128 - 2s - 16ms/step - loss: 0.3332 - mae: 0.6292 - val_loss: 0.1780 - val_mae: 0.4545 - learning_rate: 0.0052
Epoch 5/100
128/128 - 2s - 16ms/step - loss: 0.3335 - mae: 0.6294 - val_loss: 0.1861 - val_mae: 0.4705 - learning_rate: 0.0052
Epoch 6/100
128/128 - 2s - 16ms/step - loss: 0.3337 - mae: 0.6302 - val_loss: 0.1791 - val_mae: 0.4535 - learning_rate: 0.0052
Epoch 7/100
128/128 - 2s - 16ms/step - loss: 0.3328 - mae: 0.6284 - val_loss: 0.1779 - val_mae: 0.4527 - learning_rate: 0.0052
Epoch 8/100
128/128 - 2s - 16ms/step - loss: 0.3339 - 

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
- Forecast for 2025-07-22: $102970.13


In [18]:
naive_metrics, naive_pred = run_naive_model(X_btc_daily_test, 'close', 30, 'Days', 'Validation')

= Naive Model Evaluation on Validation Set (Horizon: 30 Days) =
--- Skipping Naive Model Fitting ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 9.8882%
- Directional Accuracy (DA): 42.0323%

--- Generating Final Naive Forecast ---
- Forecast for 2025-07-22: $102958.00


### GRU Model Evaluation and Final Forecast (1-Year)

In [11]:
tf.random.set_seed(42)

In [12]:
naive_metrics, naive_pred = run_naive_model(X_btc_weekly_valid, 'close', 52, 'Weeks', 'Validation')

= Naive Model Evaluation on Validation Set (Horizon: 52 Weeks) =
--- Skipping Naive Model Fitting ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 50.6933%
- Directional Accuracy (DA): 50.0000%

--- Generating Final Naive Forecast ---
- Forecast for 2025-02-09: $48295.00


In [13]:
FEATURE_COLS = ['open', 'high', 'low', 'close', 'volume']
TARGET_COL = 'close'
INPUT_WINDOW = 52
TARGET_WINDOW = 1
FORECAST_HORIZON = 52

In [14]:
gru_metrics, gru_pred = run_gru_model(X_btc_weekly_train, X_btc_weekly_valid, X_btc_weekly_test,
                                      FEATURE_COLS, TARGET_COL, FORECAST_HORIZON,
                                      INPUT_WINDOW, TARGET_WINDOW,
                                      'Days', 'Validation',
                                      naive_metrics, 'iterative')

Trial 15 Complete [00h 00m 11s]
val_mae: 0.3719090521335602

Best val_mae So Far: 0.3718875050544739
Total elapsed time: 00h 16m 00s

LSTM hyperparameter tuning complete:
- n_conv_layers: 2
- kernel_size: 3
- n_rnn_layers: 2
- use_l2: False
- dropout_rate: 0.0
- learning_rate: 0.0033710341251377745
- optimizer: adam
- clipnorm: 0.8
- filters_0: 128
- units_0: 32
- filters_1: 160
- units_1: 96
- filters_2: 160
- units_2: 128

--- Fitting GRU Model ---
Epoch 1/100
15/15 - 4s - 280ms/step - loss: 0.8131 - mae: 1.1920 - val_loss: 0.1082 - val_mae: 0.3770 - learning_rate: 0.0034
Epoch 2/100
15/15 - 0s - 15ms/step - loss: 0.4375 - mae: 0.7872 - val_loss: 0.2841 - val_mae: 0.6246 - learning_rate: 0.0034
Epoch 3/100
15/15 - 0s - 14ms/step - loss: 0.4374 - mae: 0.7945 - val_loss: 0.1871 - val_mae: 0.4676 - learning_rate: 0.0034
Epoch 4/100
15/15 - 0s - 15ms/step - loss: 0.3647 - mae: 0.6965 - val_loss: 0.1724 - val_mae: 0.4467 - learning_rate: 0.0034
Epoch 5/100
15/15 - 0s - 15ms/step - loss: 0

Walk-Forward Validation: 100%|██████████| 17/17 [01:00<00:00,  3.55s/it]


- Mean Absolute Percentage Error (MAPE): 49.4592%
- Directional Accuracy (DA): 75.0000%

--- Selecting Best GRU Model ---
- Naive Model Benchmark: MAPE: 50.6933%, DA: 50.0000%

Found 1 candidate model(s) that beat the naive model:
- Model: hyperparameters, MAPE: 49.4592%, DA: 75.0000%

--- Best GRU Model Chosen ---
- MAPE: 49.4592%
- DA: 75.0000%

Fitting, evaluating, and predicting GRU model with the best hyperparameters...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, None, 128)      │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, None, 160)      │        61,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, None, 32)       │        18,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_5           │ (None, None, 32)       │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, None, 96)       │        37,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_6           │ (None, None, 96)       │           192 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 1)        │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 120,065 (469.00 KB)

 Non-trainable params: 0 (0.00 B)

--- Fitting GRU Model ---
--- Preparing Data for GRU ---
Epoch 1/100
18/18 - 4s - 243ms/step - loss: 0.7237 - mae: 1.0855 - val_loss: 0.1999 - val_mae: 0.5326 - learning_rate: 0.0034
Epoch 2/100
18/18 - 0s - 16ms/step - loss: 0.4030 - mae: 0.7340 - val_loss: 0.1211 - val_mae: 0.3599 - learning_rate: 0.0034
Epoch 3/100
18/18 - 0s - 15ms/step - loss: 0.3766 - mae: 0.7064 - val_loss: 0.1240 - val_mae: 0.3657 - learning_rate: 0.0034
Epoch 4/100
18/18 - 0s - 13ms/step - loss: 0.3928 - mae: 0.7308 - val_loss: 0.2004 - val_mae: 0.5264 - learning_rate: 0.0034
Epoch 5/100
18/18 - 0s - 13ms/step - loss: 0.4012 - mae: 0.7352 - val_loss: 0.1201 - val_mae: 0.3623 - learning_rate: 0.0034
Epoch 6/100
18/18 - 0s - 15ms/step - loss: 0.3842 - mae: 0.7211 - val_loss: 0.1223 - val_mae: 0.3623 - learning_rate: 0.0034
Epoch 7/100
18/18 - 0s - 15ms/step - loss: 0.4264 - mae: 0.7726 - val_loss: 0.1898 - val_mae: 0.5180 - learning_rate: 0.0034
Epoch 8/100
18/18 - 0s - 14ms/step - loss: 0.3737 - mae: 0.7144 - v

Walk-Forward Validation: 100%|██████████| 18/18 [01:04<00:00,  3.60s/it]



- Mean Absolute Percentage Error (MAPE): 29.3332%
- Directional Accuracy (DA): 47.0588%

--- Generating Final GRU Forecast ---
- Forecast for 2025-08-13: $104171.10


In [15]:
naive_metrics, naive_pred = run_naive_model(X_btc_weekly_test, 'close', 52, 'Weeks', 'Test')

= Naive Model Evaluation on Test Set (Horizon: 52 Weeks) =
--- Skipping Naive Model Fitting ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 30.4386%
- Directional Accuracy (DA): 50.0000%

--- Generating Final Naive Forecast ---
- Forecast for 2026-06-21: $102958.00
